In [1]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [2]:
#hide
from fastbook import *
from IPython.display import display,HTML
from fastai.text.all import *

In [3]:
path = Path("data") #I keep a data directory in the same directory as this script
path.ls()

(#2) [Path('data/train'),Path('data/test')]

In [4]:
get_files = partial(get_text_files, folders=['train', 'test'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_files, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)

In [5]:
dls_lm.show_batch(max_n=3)

,text,text_
0,"xxbos i choose the black 5 xxmaj million xxmaj dollar xxmaj home to carry a xxmaj canon 7d with mounted 17 - 55 2.8 xxup is xxup usm . xxmaj it 's a modern and stylish bag , sort of a xxunk mini messenger . xxmaj here 's what i thought about it : interior is lined with bright light green brushed cloth . xxmaj not my favorite color -- crumpler calls it "" xxunk "" green -- but makes","i choose the black 5 xxmaj million xxmaj dollar xxmaj home to carry a xxmaj canon 7d with mounted 17 - 55 2.8 xxup is xxup usm . xxmaj it 's a modern and stylish bag , sort of a xxunk mini messenger . xxmaj here 's what i thought about it : interior is lined with bright light green brushed cloth . xxmaj not my favorite color -- crumpler calls it "" xxunk "" green -- but makes it"
1,small and the labels are hard see for anybody with a visual xxunk . xxmaj i 've placed masking tape with large - print writing to remedy reading some of the controls on the face of the xxunk may buy this radio with confidence . xxmaj the price is right for the xxunk xxmaj xxunk . xxbos xxmaj due to my error in assuming i found i needed a 58 mm lens not the 52 . xxmaj once exchanged the,and the labels are hard see for anybody with a visual xxunk . xxmaj i 've placed masking tape with large - print writing to remedy reading some of the controls on the face of the xxunk may buy this radio with confidence . xxmaj the price is right for the xxunk xxmaj xxunk . xxbos xxmaj due to my error in assuming i found i needed a 58 mm lens not the 52 . xxmaj once exchanged the lens
2,"status of the test , at xxmaj status > xxmaj device xxmaj info > xxmaj e - mail xxunk detection : xxrep 13 # xxmaj you can have the camera email you either on a schedule or if motion is detected in a configurable hot xxunk > xxmaj motion xxmaj detection … xxunk xxmaj detection : xxmaj enable … xxunk xxmaj area : click the squares of xxunk > xxmaj mail … xxunk "" enable emailing images to email ""","of the test , at xxmaj status > xxmaj device xxmaj info > xxmaj e - mail xxunk detection : xxrep 13 # xxmaj you can have the camera email you either on a schedule or if motion is detected in a configurable hot xxunk > xxmaj motion xxmaj detection … xxunk xxmaj detection : xxmaj enable … xxunk xxmaj area : click the squares of xxunk > xxmaj mail … xxunk "" enable emailing images to email "" …"


In [6]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()])

In [7]:
import time

In [8]:
start = time.time()
learn.fit_one_cycle(4, 2e-2)
end = time.time()
elapsed = end - start
print(elapsed)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.322349,4.194114,0.248094,66.294983,01:17
1,4.067857,4.089735,0.256983,59.724037,01:17
2,3.829602,4.032601,0.262444,56.407455,01:17
3,3.659336,4.019662,0.264012,55.682304,01:16


308.7860836982727


In [9]:
start = time.time()
learn.unfreeze()
learn.fit_one_cycle(1, 2e-3)
end = time.time()
elapsed = end - start
print(elapsed)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.582900,3.961968,0.272821,52.560673,01:30


90.25229072570801


#### Language Model
My original language model here has trained for a total of 11 epochs and  was overfit. Valid_loss has been rising, while training_loss has been decreasing, and accuracy decreasing. It's overfit. Looks like the sweet spot is around 4 extra epochs (total 5). Ideally, we should also fine -tune on more data. The imdb langauge model was fine tuned on 100,000 reviews, I believe, and this was barely 4K.

However, in the interest of getting once through, I will take my overfit langauge model, and train the classifier now. 

In [10]:
learn.save_encoder('finetuned')

In [11]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)

In [12]:
dls_clas.show_batch(max_n=4)

,text,category
0,"xxbos xxmaj having had a chance to spend a little time with a review model gives me a chance to share the experience with you a bit early ( before my own arrives ) . xxmaj i 'll take you hands - on with the new model , plus xxmaj i 'll share from my past two years of ipod touch ownership altogether , especially for those who have n't yet owned ( or been owned by ) one of these mobile xxunk also hidden a xxunk xxunk of info on how you can xxunk download tons of quality apps for free . xxmaj first though , let 's quickly cover what 's xxunk xxmaj faster 1ghz xxup a4 xxmaj processor - to keep up with the high demands of xxunk xxmaj ultra high resolution "" retina display "" - packs a 960 x 640 resolution at xxunk pixels per",helpful
1,"xxbos xxunk ( high definition ) xxmaj blu - ray movies played on the xxup db - d5500 are fabulous . ( standard low definition ) dvds are much clearer than played on my old xxup dvd player . xxmaj nothing more needs to be xxunk / xxup xxunk ) xxmaj the xxup bd - d5500 has a loading tray , which encourages proper handling of the disks by the edges . ( many more expensive players are "" slot xxunk ) xxmaj despite the complaint of one reviewer , it is the standard width ( 17 "" ) , exactly the same as my old xxup dvd player and my xxunk ) xxmaj the controls are illuminated , and so , easy to operate even in a dark room , but the controls are identified only by symbols . xxmaj seeing in the dark is good , feeling in",helpful
2,"xxbos xxup xxunk / 27 / xxunk i came back from xxmaj galaxy xxmaj note 10.1 because i just could not tolerate non - full xxup hd screen any more after using xxmaj infinity . i tried to convince myself it would be ok , but it was n't . xxmaj the biggest concern coming back to infinity were relatively frequent application not responding error ( anr ) and random application crash / close especially on browser , and relatively but significantly slower speed of browser page loading compared to ipad 2 or xxmaj galaxy xxmaj note 10.1 . xxmaj however , with a little tweaks i am now very satisfied with infinity as those concerns are now significantly improved to almost completely xxunk xxmaj see xxmaj stability section for added xxmaj xxunk xxmaj conclusion xxunk / 31 / xxunk xxmaj wifi related battery issue is not unit based ,",helpful
3,"xxbos xxup update 3 / 22 / 13 : xxmaj i 've used this recorder now to make hundreds of audio recordings . xxmaj its batteries have gone out many times , due to so much use . xxmaj only once did i lose some of the recording , and that only a tiny part at the end . xxmaj the sound clarity is good , even on the smallest file settings ( like xxunk ) . xxmaj am really surprised . xxmaj initial review , follows below the line . xxrep 27 - xxup upshot : right now as i type , this is the lowest - priced and best xxmaj olympus recorder in xxmaj amazon - if you buy it from xxmaj amazon . xxmaj there are other sellers of this unit advertising via xxmaj amazon who sell the 702pc for up to twice what i just paid",helpful


In [13]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy).to_fp16()

In [14]:
learn = learn.load_encoder('finetuned')

In [15]:
learn.fit_one_cycle(1, 2e-2) 

epoch,train_loss,valid_loss,accuracy,time
0,0.626118,0.616231,0.710199,00:19


In [16]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.585367,0.523512,0.756219,00:22


In [17]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.561984,0.486719,0.769901,00:31


In [18]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.479207,0.473105,0.787313,00:39


In [19]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.467911,0.473183,0.788557,00:39
